# Imports

In [1]:
import os
import re

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load Data

In [2]:
# List all data available
datapath = r'../assets'
os.listdir(datapath)

['text.md',
 'datagov_dengue.csv',
 'trend_dengue_symptoms.csv',
 '.ipynb_checkpoints',
 'jao_dengue.csv',
 'NEA_weather.csv',
 'trend_dengue.csv',
 'datagov_weather.csv',
 'NEA_dengue.csv']

Loading all data for cleaning and compiling

In [3]:
# Dengue
df_dengue_datagov = pd.read_csv(os.path.join(datapath,'datagov_dengue.csv'))
df_dengue_jao = pd.read_csv(os.path.join(datapath,'jao_dengue.csv'))
df_dengue_NEA = pd.read_csv(os.path.join(datapath,'NEA_dengue.csv'))

# Weather
df_weather_datagov = pd.read_csv(os.path.join(datapath,'datagov_weather.csv'))
df_weather_NEA = pd.read_csv(os.path.join(datapath,'NEA_weather.csv'))

# Google trend
df_trend_dengue = pd.read_csv(os.path.join(datapath,'trend_dengue.csv'))
df_trend_dengue_symp = pd.read_csv(os.path.join(datapath,'trend_dengue_symptoms.csv'))

We first clean and compile the available __dengue__ data. Most years have 52 or 53 weeks, while 2024 only has 27 weeks so far. 2019 has some missing entries which will require some imputation later.

In [4]:
# Clean and compile dengue data
df_dengue_datagov.rename(columns={'year':'Year','eweek':'Week','number':'Dengue_Count'}, inplace=True)    # Rename columns for consistency
df_dengue_datagov = df_dengue_datagov.groupby(['Year','Week']).sum()                                      # Combine normal dengue cases and hemorrhagic cases
df_dengue_datagov = df_dengue_datagov.drop(columns='type_dengue').reset_index()                           # Drop dengue fever type

df_dengue_jao.rename(columns={'Week No.':'Week', 'Dengue (Count)':'Dengue_Count'},inplace=True)           # Rename columns for consistency
df_dengue_jao.drop(columns=['Daily Rainfall Total (mm)','Mean Temperature (C)'],inplace=True)

df_dengue_NEA = df_dengue_NEA.reset_index()
df_dengue_NEA.drop(columns='Unnamed: 0',inplace=True)
df_dengue_NEA = pd.melt(df_dengue_NEA, id_vars=['index','week'],
                        value_vars=['2020','2021','2022','2023','2024'],
                        var_name='Year', value_name='Dengue_Count')                                       # Gathering into long format for consistency
df_dengue_NEA.drop(columns=['index'],inplace=True)
df_dengue_NEA.rename(columns={'week':'Week'},inplace=True)                                                # Rename columns for consistency

df_dengue = pd.concat([df_dengue_datagov,df_dengue_NEA,df_dengue_jao],axis=0)                             # Compiling all dengue datasets into single dataframe
df_dengue.dropna(inplace=True)                                                                            # Dropping small number of missing values
df_dengue = df_dengue.astype({'Year':'int', 'Dengue_Count':'int'})                                        # Converting to standard dtypes
df_dengue.drop_duplicates(subset=['Year','Week'],inplace=True)                                            # Dropping duplicate entries from different data sources
df_dengue.sort_values(by=['Year','Week'],inplace=True)                                                    # Sort in chronological order
df_dengue.groupby(['Year']).count()

,Week,Dengue_Count
Year,,
2012,52,52
2013,52,52
2014,53,53
2015,53,53
2016,53,53
2017,53,53
2018,53,53
2019,47,47
2020,53,53


Next, we clean and compile the __weather__ data. On further inpection, we see that the datagov data consists of weather information for years from 2009 to 2017 only.

In [5]:
# Show data structure and date range
display(df_weather_datagov.head(5))
pd.to_datetime(df_weather_datagov['date']).min(), pd.to_datetime(df_weather_datagov['date']).max()

,date,station,daily_rainfall_total,highest_30_min_rainfall,highest_60_min_rainfall,highest_120_min_rainfall,mean_temperature,maximum_temperature,minimum_temperature,mean_wind_speed,max_wind_speed
0,2009-01-01,Admiralty,na,na,na,na,na,na,na,na,na
1,2009-01-02,Admiralty,na,na,na,na,na,na,na,na,na
2,2009-01-03,Admiralty,na,na,na,na,na,na,na,na,na
3,2009-01-04,Admiralty,na,na,na,na,na,na,na,na,na
4,2009-01-05,Admiralty,na,na,na,na,na,na,na,na,na


(Timestamp('2009-01-01 00:00:00'), Timestamp('2017-11-30 00:00:00'))

Since we need data from 2012 to 2024, we will not use the datagov weather data, and focus on NEA weather instead, which has been scraped from NEA for the required date range. Missing data has been encoded as -1 in this dataset. As shown, the NEA dataset contains daily weather information information from multiple weather stations.

In [6]:
df_weather_NEA.sample(5)

,Unnamed: 0,Station,Year,Month,Day,Daily Rainfall Total (mm),Highest 30 min Rainfall (mm),Highest 60 min Rainfall (mm),Highest 120 min Rainfall (mm),Mean Temperature (°C),Maximum Temperature (°C),Minimum Temperature (°C),Mean Wind Speed (km/h),Max Wind Speed (km/h)
192497,26,Chai Chee,2020,5,27,35.2,24.2,29.6,31.0,-1.0,-1.0,-1.0,-1.0,-1.0
193865,23,Bukit Panjang,2021,9,24,4.6,3.4,4.6,4.6,-1.0,-1.0,-1.0,-1.0,-1.0
224693,18,Whampoa,2014,7,19,7.0,6.2,6.4,6.4,-1.0,-1.0,-1.0,-1.0,-1.0
28222,19,Clementi,2023,12,20,6.8,4.8,4.8,6.6,27.1,33.0,24.7,4.8,32.0
213430,0,Bukit Panjang,2022,2,1,0.0,0.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0


Our dengue dataset provides weekly dengue counts for the whole of Singapore. To match the dengue dataset, we take the mean values for the different weather conditions across different weather stations and over the week. We see that the highest rainfall columns consists of more than 100 missing values. These columns are dropped here for simplicity.

In [37]:
# Get week of year
df_weather_NEA_new = df_weather_NEA.replace(-1, np.nan)                                     # Encode missing values as NaN
df_weather_NEA_new.drop(columns=['Unnamed: 0'],inplace=True)                                # Drop index column
df_weather_NEA_new.dropna(subset=['Year','Month','Day'],inplace=True)                       # Dropping records where the date in missing
df_weather_NEA_new = df_weather_NEA_new.astype({'Year':'int','Month':'int','Day':'int'})    # Convert to date time and extract week of year
df_weather_NEA_new['date'] = pd.to_datetime(df_weather_NEA_new[['Year', 'Month', 'Day']])
df_weather_NEA_new['Week'] = df_weather_NEA_new['date'].dt.isocalendar().week
df_weather_NEA_new.drop(columns=['Month','Day','date','Station'],inplace=True)                        # Drop obsolete columns

# Taking mean values over all stations and over all days of a given week
df_weather_NEA_new = df_weather_NEA_new.groupby(['Year','Week']).mean()

# Check for missing values and drop
print(df_weather_NEA_new.info())
df_weather_NEA_new.drop(columns=[col for col in df_weather_NEA_new if ('Highest ' in col)], inplace=True)
df_weather_NEA_new = df_weather_NEA_new.reset_index()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 654 entries, (np.int64(2012), np.uint32(1)) to (np.int64(2024), np.uint32(26))
Data columns (total 9 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Daily Rainfall Total (mm)      654 non-null    float64
 1   Highest 30 min Rainfall (mm)   550 non-null    float64
 2   Highest 60 min Rainfall (mm)   550 non-null    float64
 3   Highest 120 min Rainfall (mm)  550 non-null    float64
 4   Mean Temperature (°C)          654 non-null    float64
 5   Maximum Temperature (°C)       654 non-null    float64
 6   Minimum Temperature (°C)       654 non-null    float64
 7   Mean Wind Speed (km/h)         654 non-null    float64
 8   Max Wind Speed (km/h)          654 non-null    float64
dtypes: float64(9)
memory usage: 47.7 KB
None


The final dataframe to be used as weather input is as shown below.

In [39]:
df_weather_NEA_new.sample(5)

,Year,Week,Daily Rainfall Total (mm),Mean Temperature (°C),Maximum Temperature (°C),Minimum Temperature (°C),Mean Wind Speed (km/h),Max Wind Speed (km/h)
156,2015,1,0.143103,26.950000,30.065789,24.664474,10.061111,32.661111
381,2019,16,2.888235,29.468932,33.786364,26.536364,7.766667,30.047009
49,2012,50,17.119048,26.356429,30.723571,24.045714,5.627857,31.302143
362,2018,49,9.517280,27.255238,31.716071,24.834821,5.783929,26.871429
126,2014,23,6.938407,28.437857,31.353571,26.179286,7.195714,31.436429


Finally, __google search trends__ regarding dengue may indicate imminent occurence of dengue clusters. Specifically, we include search trends related to the keywords "dengue" and "dengue+symptoms".

In [40]:
# Google trend
df_trend_dengue
# df_trend_dengue_symp

,Category: All categories
Month,dengue: (Singapore)
2012-01,10
2012-02,10
2012-03,9
2012-04,10
...,...
2024-03,18
2024-04,24
2024-05,17
2024-06,17


In [34]:
df_weather_NEA_new

Daily Rainfall Total (mm)  Mean Temperature (°C)  \
Year Week                                                     
2012 1                     10.266250              26.866026   
     2                      3.740000              26.434783   
     3                     13.200000              27.238235   
     4                      1.522381              26.784672   
     5                      7.472143              26.544286   
...                              ...                    ...   
2024 22                     3.380634              29.165421   
     23                     4.713805              28.941739   
     24                     8.764846              28.844828   
     25                    12.723232              28.141880   
     26                     9.580731              28.566387   

           Maximum Temperature (°C)  Minimum Temperature (°C)  \
Year Week                                                       
2012 1                    30.521875                 24.589375   
     2                    29.797143                 23.925714   
     3                    31.835000                 24.474286   
     4                    30.781429                 24.417857   
     5                    30.213571                 24.176429   
...                             ...                       ...   
2024 22                   32.323364                 26.918692   
     23                   32.826957                 26.196522   
     24                   32.706897                 25.794828   
     25                   31.214530                 25.094872   
     26                   32.085714                 25.927731   

           Mean Wind Speed (km/h)  Max Wind Speed (km/h)  
Year Week                                                 
2012 1                  10.653125              36.228750  
     2                   8.486429              33.596429  
     3                   6.683571              29.836429  
     4                   9.255714              33.138571  
     5                   9.028571              33.248571  
...                           ...                    ...  
2024 22                  6.630108              31.682796  
     23                  6.082178              28.440594  
     24                  5.838235              29.777451  
     25                  7.950485              38.554369  
     26                  6.776190              31.419048  

[654 rows x 6 columns]

In [41]:
display(df_dengue_datagov.sample(5))

,year,eweek,type_dengue,number
474,2018,26,Dengue,75.0
353,2017,18,DHF,0.0
520,2018,49,Dengue,113.0
307,2016,48,DHF,0.0
141,2015,18,DHF,0.0


In [43]:
display(df_dengue_jao.sample(5))

,Year,Week No.,Daily Rainfall Total (mm),Mean Temperature (C),Dengue (Count)
131,2014,28,11.465000,28.110034,819
383,2019,19,10.717972,28.814868,287
117,2014,14,5.524933,28.351839,240
64,2013,13,9.503841,28.433556,316
206,2015,50,1.190857,27.757774,333


In [45]:
display(df_dengue_NEA.sample(5))

,Unnamed: 0,week,2020,2021,2022,2023,2024
23,23,24,1153.0,136.0,1254.0,162.0,303.0
12,12,13,377.0,114.0,530.0,128.0,349.0
22,22,23,867.0,137.0,1452.0,111.0,333.0
42,42,43,564.0,70.0,303.0,208.0,NaN
3,3,4,309.0,137.0,166.0,273.0,430.0
